In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import datetime
import json
import numpy as np
import os
import pandas as pd
import sys 
import tensorflow as tf
import time
import matplotlib.pyplot as plt

from keras.callbacks import History 
history = History()

BATCH_SIZE = 10

MAX_number_of_attributes = 39

print(tf.__version__)

2.7.0


In [3]:
path_training ="/content/drive/MyDrive/IC-Colab-CSV files/01_Atributos_Weve.csv"
path_test ="/content/drive/MyDrive/IC-Colab-CSV files/02_Atributos_Weve.csv"
df_raw = pd.read_csv(path_training)
df_test_raw = pd.read_csv(path_test)
df_raw
target = df_raw.pop('target')
server_port = df_raw.pop('ServerPort')
client_port = df_raw.pop('ClientPort')

target_test = df_test_raw.pop('target')
server_port_test = df_test_raw.pop('ServerPort')
client_port = df_test_raw.pop('ClientPort')


In [4]:
array_de_atributos = ['60', '59', '82', '46', '17', '84', '79', '10', '90', '15', '24', '92', '91', '83', '48', '37', '22', '88', '47', '45', '34', '32', '36', '38', '33', '31', '80', '61', '62', '64', '89', '63', '29', '87', '81', '75','76','77','78']

                     

In [5]:
classes = ['WWW','MAIL','FTP-CONTROL','FTP-PASV','ATTACK','P2P','DATABASE','FTP-DATA','MULTIMEDIA','SERVICES','INTERACTIVE','GAMES']
#classes = list(target.unique())
#classes
#Converts the above list of classes to an array of index
target = target.apply(lambda x: classes.index(x))
target.unique()
target_test = target_test.apply(lambda x: classes.index(x))

In [9]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(10, activation='relu'), 
    tf.keras.layers.Dense(12)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [12]:
  model = get_basic_model()

ValueError: ignored

In [11]:

for i in range(MAX_number_of_attributes):
  number_of_attributes = i
  df = df_raw[array_de_atributos[0:number_of_attributes]]
  df_test = df_test_raw[array_de_atributos[0:number_of_attributes]]
  #df

  numeric_feature_names = array_de_atributos[0:number_of_attributes]
  numeric_features = df[numeric_feature_names] # Data for training
  numeric_features_test = df_test[numeric_feature_names] # Data for testing
  #numeric_features.head()

  #converting pandas dataframe to tensorflow object
  numeric_features = tf.convert_to_tensor(numeric_features)
  numeric_features_test = tf.convert_to_tensor(numeric_features_test)
  #numeric_features

  # Normalize the data
  normalizer = tf.keras.layers.Normalization(axis=-1)
  normalizer.adapt(numeric_features)


  history = model.fit(numeric_features, target, epochs=10, batch_size=BATCH_SIZE, callbacks=[history])

  train_accuracy = history.history['accuracy']
  test_loss, test_acc = model.evaluate(numeric_features_test,  target_test, verbose=2, batch_size=BATCH_SIZE) 
  #print('\nTest accuracy:', test_acc)



ValueError: ignored

Ideia: Fazer a normalização entre 0 e 1 conforme o Andrews, aqui atualmente ele está centrado em 0 com desvio padrão em 1

In [ ]:
#layer0_weights = model.layers[0].get_weights()[0] #weight
#layer0_biases  = model.layers[0].get_weights()[1] #bias
layer1_weights = model.layers[1].get_weights()[0] #weight
#layer1_biases  = model.layers[1].get_weights()[1] #bias
#print('Layer 1 weights:',layer1_weights)
# print('\n')

weight_attribute = []
for attr_i, weights in enumerate(layer1_weights):
  weight_attribute.append((weights.sum(), numeric_feature_names[attr_i]))

weight_attribute = sorted(weight_attribute)

print(weight_attribute)

attributes = [x for _, x in weight_attribute]

print(attributes)

In [ ]:
current_date_and_time = datetime.date.today()
current_date_and_time_string = str(current_date_and_time)

report = {
    "datetime": current_date_and_time_string,
    "training_dataset_shape":  df.shape,
    "test_dataset_shape": df_test.shape,
    "attributes": array_de_atributos[0:df.shape[1]],
    "number_of_attributes": number_of_attributes,
    "accuracy_train": train_accuracy[-1],
    "accuracy_test": test_acc
}

titulo = "/content/drive/MyDrive/IC-Skynet/NNreports/%s"%current_date_and_time_string + " With %d Attributes.json"%number_of_attributes
titulo

with open(titulo, "w") as f:
  json.dump(report, f)

with open(titulo, "r") as f:
  rep = json.load(f)

display(rep)

#print(rep["datetime"])


